<h1> 📘 Biofilter — Reports to ETL Management </h1>

Biofilter provides a set of built-in ETL reports designed to help users monitor, inspect, and debug data ingestion pipelines. These reports expose the execution status, history, and metadata of ETL processes in a structured and reproducible way, making it easier to understand what has been loaded, what is currently running, and where failures may have occurred.

Methods

* report.run() — Execute a report and return the results as a pandas DataFrame.

Available Reports

* etl_status — High-level, consolidated view of ETL pipelines, showing the latest successful execution per data source and overall pipeline health.

* etl_packages — Detailed, row-level view of all ETL packages, exposing extract, transform, and load steps, timestamps, statuses, and logs for debugging and auditing.

--------

### Start Biofilter3R

In [1]:
from biofilter import Biofilter
bf = Biofilter()

[INFO] ════════════════════════════════════
[INFO] 🚀 Initializing Biofilter3R
[INFO]    • Version: 3.2.0
[INFO]    • Debug mode: False
[INFO]    • Config: /home/bioadmin/biofilter/.biofilter.toml
[INFO] ════════════════════════════════════
[INFO] 🔌 Database connection established
[INFO]    • Engine: postgresql+psycopg2
[INFO]    • Host:   localhost
[INFO]    • DB:     biofilter
[INFO]    • Time:   0.6 ms
[INFO] ════════════════════════════════════


------

### 1. Report to extract ETL Status in the Database

Returns a consolidated ETL status view per data source, showing the latest successful pipeline state (Extract → Transform → Load) for each selected data source.

This report is designed to answer the question:

* For these data sources, do we have a valid and up-to-date ETL pipeline?

In [2]:
bf.report.explain("etl_status")

📦 ETL Status (Latest Good)

This report summarizes ETL execution health per DataSource by selecting:
- The most recent GOOD extract package (completed or up-to-date)
- The most recent GOOD transform package
- The most recent GOOD load package

If the latest extract is newer but transform/load are missing or not aligned
(by hash), the report still shows the last good transform/load and flags them
as stale (not aligned with latest extract).



In [2]:
# df = bf.report.run("report_etl_status")
df = bf.report.run(
    "etl_status",
    source_system="ncbi",
    data_sources=["dbsnp_chr6",],
)
df

,source_system,data_source,data_type,pipeline_ok,extract_package_id,extract_status,extract_end,transform_package_id,transform_status,transform_end,transform_aligned_with_latest_extract,load_package_id,load_status,load_end,load_aligned_with_latest_transform,latest_error
0,NCBI,dbsnp_chr6,Variant,False,84,completed,2025-12-14 00:16:48.905348,85,completed,2025-12-14 20:38:19.047399,True,None,None,None,False,None


-------

### 2. Report to extract ETL the Packages history

The etl_packages report provides a detailed, row-level view of ETL execution history, returning one row per ETLPackage created in the system. It exposes full metadata for each package, including source system, data source, operation type (extract, transform, load), execution timestamps, per-stage statuses, hashes, row counts, and optional logs or error messages. This report is intended for debugging, auditing, and operational analysis, allowing users to inspect failed, running, skipped, or completed ETL steps, trace pipeline execution order, and understand exactly what happened during each ETL phase. Unlike etl_status, it does not consolidate results and instead reflects the raw ETL lifecycle as recorded in the database.

In [4]:
bf.report.explain("etl_packages")

📦 ETL Packages – Detailed Audit Report

This report provides a **raw, non-aggregated** view of the ETL execution state.

Each row corresponds to **one ETLPackage record**, which may represent:
- one ETL stage (extract / transform / load), or
- one full execution attempt, depending on how the ETL was triggered.

The report joins:
- ETLSourceSystem (e.g. NCBI, Ensembl, UniProt)
- ETLDataSource (e.g. dbSNP_chr1, ensembl, hgnc)
- ETLPackage (execution metadata)

This report is intentionally *not consolidated*.
It is designed for:
- Debugging failed or stuck jobs
- Auditing execution history
- Understanding how many packages were created per data source
- Verifying status transitions across ETL stages

Recommended usage:
- Use this report to identify inconsistencies
- Fix ETL status logic
- Only then create consolidated / dashboard-style reports



In [9]:
# df = bf.report.run("etl_packages")
df = bf.report.run(
    "etl_packages",
    source_system="ncbi",
    # data_sources=["dbsnp_chr6",],
)
df

,package_id,created_at,source_system,data_source,status,operation_type,version_tag,note,log,extract_status,...,transform_rows,transform_hash,load_status,load_start,load_end,load_rows,load_hash,extract_minutes,transform_minutes,load_minutes
0,111,2025-12-16 06:08:26.692916,NCBI,dbsnp_chr10,running,running,None,None,"{""note"": ""load skipped"", ""reason"": ""no transfo...",pending,...,None,None,not-applicable,2025-12-16 06:08:26.743429,2025-12-16 06:08:26.743487,None,None,NaN,NaN,9.666667e-07
1,110,2025-12-16 05:20:50.371078,NCBI,dbsnp_chr9,running,running,None,None,"{""note"": ""load skipped"", ""reason"": ""no transfo...",pending,...,None,None,not-applicable,2025-12-16 05:20:50.431777,2025-12-16 05:20:50.431832,None,None,NaN,NaN,9.166667e-07
2,98,2025-12-15 15:46:22.126611,NCBI,dbsnp_chr9,failed,transform,None,None,None,not-applicable,...,None,981dcd9b31538d6f5af5701bd5c60dfe,not-applicable,NaT,NaT,None,None,NaN,814.465810,NaN
3,97,2025-12-15 15:41:31.872640,NCBI,dbsnp_chr10,failed,transform,None,None,None,not-applicable,...,None,6d996161d665e139d0c72041ae825ce7,not-applicable,NaT,NaT,None,None,NaN,866.909423,NaN
4,96,2025-12-15 14:18:11.257703,NCBI,dbsnp_chr10,completed,extract,None,None,None,completed,...,None,None,pending,NaT,NaT,None,None,83.334922,NaN,NaN
5,95,2025-12-15 14:17:58.342017,NCBI,dbsnp_chr9,completed,extract,None,None,None,completed,...,None,None,pending,NaT,NaT,None,None,88.386621,NaN,NaN
6,94,2025-12-14 22:58:12.935550,NCBI,dbsnp_chr7,completed,load,None,None,None,not-applicable,...,None,None,completed,2025-12-14 22:58:12.987173,2025-12-16 10:44:42.763483,None,e3185931e6159b31bdf3f50f55973ff3,NaN,NaN,2.146496e+03
7,93,2025-12-14 21:42:07.757679,NCBI,dbsnp_chr8,completed,load,None,None,None,not-applicable,...,None,None,completed,2025-12-14 21:42:07.782899,2025-12-16 08:10:35.110971,None,2ce6dd8b7b4c844d90ec917eb2743fed,NaN,NaN,2.068455e+03
8,92,2025-12-14 20:38:19.081557,NCBI,dbsnp_chr6,failed,load,None,None,None,not-applicable,...,None,None,not-applicable,2025-12-14 20:38:19.119724,NaT,None,2b656149c86f27edd02ec231fbbf0cb4,NaN,NaN,NaN
9,91,2025-12-14 20:22:00.497338,NCBI,dbsnp_chr5,completed,load,None,None,None,not-applicable,...,None,None,completed,2025-12-14 20:22:00.533792,2025-12-16 09:36:34.092903,None,f7082da928d66bf24c3694a48213add4,NaN,NaN,2.234559e+03
